In [23]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from rff.layers import GaussianEncoding #pip install random-fourier-features-pytorch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import os
import optuna
from optuna.trial import TrialState
from tab_transformer_pytorch import FTTransformer

In [2]:
# Run regardless if you do or do not have GPU so all tensors are moved to right location later on
if torch.cuda.is_available():
    device_in_use = torch.device("cuda")
    print("GPU is available and being used")
else:
    device_in_use = torch.device("cpu")
    print("GPU is not available, using CPU instead")

# LOAD AND PROCESS DATA
**EXAMPLE WITH Income**

In [3]:
# df_train = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/income/train.csv')
# df_test = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/income/test.csv')
# df_val = pd.read_csv('/home/cscadmin/CyberResearch/CAT-Transformer/datasets/income/validation.csv') #READ FROM RIGHT SPOT

df_train = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\income\train.csv')
df_test = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\income\test.csv')
df_val = pd.read_csv(r'C:\Users\smbm2\projects\CAT-Transformer\datasets\income\validation.csv') #READ FROM RIGHT SPOT

#Take a look at what the datasets look like initially to get an idea
df_train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [4]:
cont_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
cat_columns = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country']
target = ['income']

#CHECKING TO MAKE SURE YOUR LIST IS CORRECT (NO NEED TO TOUCH)
yourlist = cont_columns + cat_columns+target
yourlist.sort()
oglist = list(df_train.columns)
oglist.sort()

assert(yourlist == oglist), "You may of spelled feature name wrong or you forgot to put on of them in the list"


In [5]:
#Get class counts and store in a list below

for x in cat_columns:
    print(max(len(df_train[x].value_counts()), len(df_val[x].value_counts()), len(df_test[x].value_counts())))

10
16
7
16
6
5
2
43


In [6]:
# Manually enter class counts in the same order
cat_features = (10,16,7,16,6,5,2,43)

In [7]:
#Getting the number of classes in your classification target
target_classes = [max(len(df_train[target].value_counts()), len(df_val[target].value_counts()),len(df_test[target].value_counts()))]
target_classes = target_classes[0]

In [8]:
# Create a StandardScaler and fit it to the cont features
scaler = StandardScaler()
scaler.fit(df_train[cont_columns])

# Transform the training, test, and validation datasets
df_train[cont_columns] = scaler.transform(df_train[cont_columns])
df_test[cont_columns] = scaler.transform(df_test[cont_columns])
df_val[cont_columns] = scaler.transform(df_val[cont_columns])

In [9]:
class SingleTaskDataset(Dataset):
    def __init__(self, df : pd.DataFrame, cat_columns, num_columns,task1_column):
        self.n = df.shape[0]
        
        self.task1_labels = df[task1_column].astype(np.float32).values

        self.cate = df[cat_columns].astype(np.int64).values
        self.num = df[num_columns].astype(np.float32).values


    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        # Retrieve features and labels from the dataframe using column names
        cat_features = self.cate[idx]
        num_features = self.num[idx]
        labels_task1 = self.task1_labels[idx]

        return cat_features, num_features, labels_task1

#Wrapping in Dataset
train_dataset = SingleTaskDataset(df_train, cat_columns, cont_columns, 'income')
val_dataset = SingleTaskDataset(df_val, cat_columns, cont_columns, 'income')
test_dataset = SingleTaskDataset(df_test, cat_columns, cont_columns, 'income')

#This is a hyperparameter that is not tuned. Maybe mess with what makes sense here
#Also try looking to see what other papers have done
batch_size = 256

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
for cat, cont, label in train_dataloader:
    print(cat.shape)
    print(cont.shape)
    print(label.shape)

torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])


torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6])
torch.Size([256])
torch.Size([256, 8])
torch.Size([256, 6]

In [28]:
# each task loss is scaled by its own learnable parameter, then regularization is applied 
class LossFunctions(nn.Module):
    def __init__(self, num_tasks):
        super(LossFunctions, self).__init__()
        self.num_tasks = num_tasks

        self.loss_fns = [nn.CrossEntropyLoss() for x in range(num_tasks)] 

    def forward(self, predictions, labels_task1):

        #task 1
        target = labels_task1.long()
        prediction = predictions
        loss_fn = self.loss_fns[0]
        task_loss = loss_fn(prediction, target)
        
        return task_loss
    
# Training and Testing Loops
def train(dataloader, model, loss_function, optimizer, device_in_use):
    model.train()

    total_loss = 0
    total_correct_1 = 0
    total_samples_1 = 0
    all_targets_1 = []
    all_predictions_1 = []

    for (categorical, numerical, labels_task1) in dataloader:
        categorical, numerical, labels_task1 = categorical.to(device_in_use), numerical.to(device_in_use), labels_task1.to(device_in_use)
        
        #running them through model and modifying the shape slightly for the loss function
        task_predictions = model(categorical, numerical)
        task_predictions = task_predictions.squeeze(1)
        
        loss = loss_function(task_predictions, labels_task1)
        total_loss += loss.item()
        
        #computing accuracy for first target
        y_pred_softmax_1 = torch.softmax(task_predictions, dim=1)
        _, y_pred_labels_1 = torch.max(y_pred_softmax_1, dim=1)
        total_correct_1 += (y_pred_labels_1 == labels_task1).sum().item()
        total_samples_1 += labels_task1.size(0)
        all_targets_1.extend(labels_task1.cpu().numpy())
        all_predictions_1.extend(y_pred_labels_1.cpu().numpy())


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss/len(dataloader)
    accuracy_1 = total_correct_1 / total_samples_1

    return avg_loss, accuracy_1

def test(dataloader, model, loss_function, device_in_use):
    model.eval()
    total_loss = 0
    
    total_loss = 0
    total_correct_1 = 0
    total_samples_1 = 0
    all_targets_1 = []
    all_predictions_1 = []

    with torch.no_grad():
        for (categorical, numerical, labels_task1) in dataloader:
            categorical, numerical, labels_task1 = categorical.to(device_in_use), numerical.to(device_in_use), labels_task1.to(device_in_use)
            
            #running them through model and modifying the shape slightly for the loss function
            task_predictions = model(categorical, numerical)
            task_predictions = task_predictions.squeeze(1)
            loss = loss_function(task_predictions, labels_task1)
            total_loss += loss.item()
            
            #computing accuracy for first target
            y_pred_softmax_1 = torch.softmax(task_predictions, dim=1)
            _, y_pred_labels_1 = torch.max(y_pred_softmax_1, dim=1)
            total_correct_1 += (y_pred_labels_1 == labels_task1).sum().item()
            total_samples_1 += labels_task1.size(0)
            all_targets_1.extend(labels_task1.cpu().numpy())
            all_predictions_1.extend(y_pred_labels_1.cpu().numpy())

    avg = total_loss/len(dataloader)
    accuracy_1 = total_correct_1 / total_samples_1
    f1_1 = f1_score(all_targets_1, all_predictions_1, average='weighted')

    return avg, accuracy_1, all_predictions_1, all_targets_1, f1_1

def format_metric(value): # Used to format the metrics output
    return f"{value:.4f}"


# RUN EXPERIMENTS

1. Using Optuna to optimize FT-Transformers hyperparameters for your dataset

In [26]:
# Define the early stopping mechanism
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_metric = float('-inf')
        self.early_stop = False

    def __call__(self, metric):
        if metric > self.best_metric:
            self.best_metric = metric
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Function to log results to a text file
def log_to_file(filename, text):
    with open(filename, 'a') as f:
        f.write(text + '\n')

def objective(trial):
    trial_number = trial.number
    '''
        attn_dropout = 0.,
        ff_dropout = 0.
    '''

    # Define hyperparameters to search over
    depth = trial.suggest_int('depth', 1, 2)
    # Ensure that embed_size is divisible by num_layers
    dim = trial.suggest_categorical("dim", [50, 60, 70, 80, 90, 100, 120, 140, 160])
    heads = trial.suggest_categorical("heads", [1, 5, 10])
    dim_head = trial.suggest_categorical("dim_head", [5,10,15,20,25,30])
    attn_dropout = trial.suggest_categorical('attn_dropout', [0,.1,.2,.5])
    ff_dropout = trial.suggest_categorical('ff_dropout', [0,.1,.2,.5])

    learning_rate = trial.suggest_categorical('learning_rate', [0.0001, 0.001, 0.01])

    num_epochs = 100

    # Create your model with the sampled hyperparameters
    model = FTTransformer(categories=cat_features,
                          num_continuous=len(cont_columns),
                          dim=dim,
                          depth=depth,
                          heads=heads,
                          dim_head=dim_head,
                          dim_out=target_classes,
                          attn_dropout=attn_dropout,
                          ff_dropout=ff_dropout).to(device_in_use)

    # Define loss function and optimizer
    loss_function = LossFunctions(1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=10)  # Adjust patience as needed

    # Training loop with a large number of epochs
    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(train_dataloader, model, loss_function, optimizer, device_in_use)
        
        # Validation loop
        val_loss, val_accuracy, _, _, _ = test(val_dataloader, model, loss_function, device_in_use)
        
        # Check if we should early stop based on validation accuracy
        if early_stopping(val_accuracy):
            break

    
    # Log the final test accuracy for this trial to a shared log file
    final_log = f"Trial {trial_number} completed. Validation Accuracy = {val_accuracy:.4f}"
    log_to_file('all_trials_log.txt', final_log)

    # Return the test accuracy as the objective to optimize
    return val_accuracy

In [29]:
# Set the number of optimization trials
num_trials = 50

# Create an Optuna study
study = optuna.create_study(direction='maximize')  # Maximize validation accuracy

# Start the optimization process
study.optimize(objective, n_trials=num_trials, show_progress_bar=True)

# Get the best hyperparameters and the validation accuracy at the point of early stopping
best_params = study.best_params
best_val_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy (at Early Stopping):", best_val_accuracy)

[I 2023-10-31 16:06:38,591] A new study created in memory with name: no-name-58026bf1-b804-4745-a0ad-84a7cfbf139e
Best trial: 0. Best value: 0.847978: 100%|██████████| 1/1 [00:36<00:00, 36.05s/it]

[I 2023-10-31 16:07:14,641] Trial 0 finished with value: 0.847978160723426 and parameters: {'depth': 2, 'dim': 60, 'heads': 5, 'dim_head': 10, 'attn_dropout': 0.1, 'ff_dropout': 0, 'learning_rate': 0.0001}. Best is trial 0 with value: 0.847978160723426.
Best Hyperparameters: {'depth': 2, 'dim': 60, 'heads': 5, 'dim_head': 10, 'attn_dropout': 0.1, 'ff_dropout': 0, 'learning_rate': 0.0001}
Best Validation Accuracy (at Early Stopping): 0.847978160723426


In [30]:
#Testing against the test dataset
model = FTTransformer(categories=cat_features,
                          num_continuous=len(cont_columns),
                          dim=best_params['dim'],
                          depth=best_params['depth'],
                          heads=best_params['heads'],
                          dim_head=best_params['dim_head'],
                          dim_out=target_classes,
                          attn_dropout=best_params['attn_dropout'],
                          ff_dropout=best_params['ff_dropout']).to(device_in_use)
loss_functions = LossFunctions(1)
optimizer = torch.optim.Adam(params=model.parameters(), lr = best_params['learning_rate']) # Maybe try messing around with optimizers. try other torch optimizers with different configurations.
epochs = 100 #Set the number of epochs
early_stopping = EarlyStopping(patience=5)

train_losses = []
train_accuracies_1 = [] 
train_accuracies_2 = []
train_recalls = [] 
train_f1_scores = [] 
test_losses = []
test_accuracies_1 = []
test_accuracies_2 = []
test_recalls = []  
test_f1_scores = [] 
all_attention_scores = []

for t in range(epochs):
  train_loss, train_accuracy_1 = train(train_dataloader, model, loss_functions, optimizer, device_in_use=device_in_use)
  test_loss, test_accuracy_1, all_predictions_1, all_targets_1, f1_1 = test(test_dataloader, model, loss_functions, device_in_use=device_in_use)
  train_losses.append(train_loss)
  train_accuracies_1.append(train_accuracy_1)
  # train_accuracies_2.append(train_accuracy_2)
  # train_recalls.append(train_recall) 
  # train_f1_scores.append(train_f1)
  test_losses.append(test_loss)
  test_accuracies_1.append(test_accuracy_1)
  # test_accuracies_2.append(test_accuracy_2)
  # test_recalls.append(test_recall)
  # test_f1_scores.append(test_f1)
  # Formatting for easier reading
  epoch_str = f"Epoch [{t+1:2}/{epochs}]"
  train_metrics = f"Train: Loss {format_metric(train_loss)}, Accuracy {format_metric(train_accuracy_1)}"
  test_metrics = f"Test: Loss {format_metric(test_loss)}, Accuracy {format_metric(test_accuracy_1)}, F1 {format_metric(f1_1)}"
  print(f"{epoch_str:20} | {train_metrics:65} | {test_metrics}")

  if early_stopping(test_accuracy_1):
    break

torch.save(model.state_dict(), 'final_model_trained.pth')

# Plotting the loss curves
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), [l for l in test_losses], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Curve')
plt.legend()


Epoch [ 1/75]        | Train: Loss 0.5173, Accuracy 0.7501                               | Test: Loss 0.3896, Accuracy 0.8288, F1 0.8175
Epoch [ 2/75]        | Train: Loss 0.3476, Accuracy 0.8405                               | Test: Loss 0.3413, Accuracy 0.8427, F1 0.8438


KeyboardInterrupt: 